# import the libraries required

In [4]:
import numpy as np
import pandas as pd


# read the csv file

In [5]:
a_csv=pd.read_csv('5_a.csv')
print(a_csv.head(10))
a_csv.shape
        

     y     proba
0  1.0  0.637387
1  1.0  0.635165
2  1.0  0.766586
3  1.0  0.724564
4  1.0  0.889199
5  1.0  0.601600
6  1.0  0.666323
7  1.0  0.567012
8  1.0  0.650230
9  1.0  0.829346


(10100, 2)

# change the prob values to binary values.using thresholds.

In [6]:
df=a_csv['proba']

In [7]:
print(len(df))


10100


# convert probabilistic values to either 0 or 1 based on threshold

In [8]:
count=0
count1=0
for i in range(0,len(df)):
    if df.loc[i]>0.5:
        count+=1
        df=df.replace(df.loc[i],1)
    elif df.loc[i]<=0.5:
        count1+=1
        df=df.replace(df.loc[i],0)
print(df.head(10))
print('positive points count',count)
print('positive points count',count1)

0    1.0
1    1.0
2    1.0
3    1.0
4    1.0
5    1.0
6    1.0
7    1.0
8    1.0
9    1.0
Name: proba, dtype: float64
positive points count 10100
positive points count 0


In [9]:
df.describe()

count    10100.0
mean         1.0
std          0.0
min          1.0
25%          1.0
50%          1.0
75%          1.0
max          1.0
Name: proba, dtype: float64

# create column in the dataset to store yhat values

In [10]:
a_csv['yhat']=df.values

In [11]:
a_csv.head(10)

,y,proba,yhat
0,1.0,0.637387,1.0
1,1.0,0.635165,1.0
2,1.0,0.766586,1.0
3,1.0,0.724564,1.0
4,1.0,0.889199,1.0
5,1.0,0.601600,1.0
6,1.0,0.666323,1.0
7,1.0,0.567012,1.0
8,1.0,0.650230,1.0
9,1.0,0.829346,1.0


# calculate total number of positive and negative points in the dataset

In [12]:
pos=0
neg=0
def sudhi():
    global pos
    global neg
    for i in range(0,len(a_csv)):
        if a_csv['y'].loc[i]==1:
            pos+=1
            
        elif a_csv['y'].loc[i]==0:
            neg+=1
    return pos,neg
sudhi()
print('positive points count(p)',pos)
print('negative points count(n)',neg)
    

positive points count(p) 10000
negative points count(n) 100


In [13]:
print(pos)
print(neg)

10000
100


# observations

1)there are 100 negative points (0) in 'y' that have been  falsely classifed as positive points(1) in yhat.

# CONFUSION MATRIX CALCULATION

In [14]:
confusion_matrix=[]
TN=0
FN=0
FP=0
TP=0
for i in range(0,len(a_csv)):
    global TN
    global TP
    global FP
    global FN
    if a_csv['y'].loc[i]==a_csv['yhat'].loc[i]:
        
        if a_csv['y'].loc[i]==0:
            
            TN+=1
            
        elif a_csv['y'].loc[i]==1:
            
            TP+=1
            
    elif a_csv['y'].loc[i]!=a_csv['yhat'].loc[i]:
        
        if a_csv['y'].loc[i]==0 and a_csv['yhat'].loc[i]==1:
            FP+=1
        elif a_csv['y'].loc[i]==1 and a_csv['yhat'].loc[i]==0:
            FN+=1
            
print('TP',TP)
print('TN',TN)
print('FP',FP)
print('FN',FN)
confusion_matrix.append(TN)
confusion_matrix.append(FN)
confusion_matrix.append(FP)
confusion_matrix.append(TP)

x=np.reshape(confusion_matrix,(2, 2))
#print(confusion_matrix)
print('CONFUSION MATRIX \n',x)

TP 10000
TN 0
FP 100
FN 0
CONFUSION MATRIX 
 [[    0     0]
 [  100 10000]]


# PRECISION , RECALL , F1-SCORE calculation

In [15]:
precision=((TP)/(TP+FP))
recall=((TP)/(TP+FN))
print('precision \n',precision)
print('recall \n',recall)
F1_score=2*(precision*recall)/(precision+recall)
print('F1-score \n',F1_score)


precision 
 0.9900990099009901
recall 
 1.0
F1-score 
 0.9950248756218906


# ACCURACY SCORE

In [16]:
accuracy_score=(TP+TN)/(TP+FP+FN+TN)
print('accuracy score \n',accuracy_score)

accuracy score 
 0.9900990099009901


# COMPUTE AUC by considering each probability value in yhat as threshold and compute TPR and FPR

In [30]:
tpr_lst=[]
fpr_lst=[]
fn_lst=[]
tn_lst=[]
from tqdm import tqdm
a_csv=pd.read_csv('5_a.csv')
#a_csv['proba']=sorted(a_csv['proba'])
sorted_data=a_csv.sort_values('proba',ascending=True)
for threshold in tqdm(sorted_data['proba']):
    y_hat=[]
    for value in sorted_data['proba']:
        if (value<=threshold):
            y_hat.append(0.0)
        else:
            y_hat.append(1.0)
    #print(y_hat[:10])
    sorted_data['y_pred']=y_hat
    #print(a_csv.head(10))
    for k in a_csv:
        tp = (((sorted_data['y'])==1.0) & ((sorted_data['y_pred']) == 1.0)).sum()   
        fp = (((sorted_data['y'])==0.0) & ((sorted_data['y_pred']) == 1.0)).sum()
        tn=(((sorted_data['y'])==0.0) & ((sorted_data['y_pred']) == 0.0)).sum()
        fn=(((sorted_data['y'])==1.0) & ((sorted_data['y_pred']) == 0.0)).sum()

    tpr_lst.append(tp/(tp+fn))
    fpr_lst.append(fp/(fp+tn))
    #print('TPR\n',tpr_lst[:])
    
    #print('FPR\n',fpr_lst[:])
    
    tn_lst.append(tn)
    fn_lst.append(fn)
x=sorted(tpr_lst)
y=sorted(fpr_lst)
auc = np.trapz(x,y)

print('AUC score is : ',auc)
print('TP',tp)
print('FP',fp)
print('TN',tn)
print('FN',fn)







  0%|                                                                                        | 0/10100 [00:00<?, ?it/s]





  0%|                                                                                | 9/10100 [00:00<01:56, 86.57it/s]





  0%|▏                                                                              | 19/10100 [00:00<01:52, 89.98it/s]





  0%|▏                                                                              | 28/10100 [00:00<01:52, 89.47it/s]





  0%|▎                                                                              | 38/10100 [00:00<01:49, 92.14it/s]





  0%|▍                                                                              | 48/10100 [00:00<01:47, 93.85it/s]





  1%|▍                                                                              | 59/10100 [00:00<01:43, 96.90it/s]





  1%|▌                                                                              | 70/10100 [00:00<01:40, 99.68it/s]

 14%|██████████▊                                                                 | 1437/10100 [00:13<01:22, 105.37it/s]





 14%|██████████▉                                                                 | 1448/10100 [00:13<01:21, 106.12it/s]





 14%|███████████                                                                  | 1459/10100 [00:14<01:27, 98.89it/s]





 15%|███████████                                                                 | 1470/10100 [00:14<01:25, 101.43it/s]





 15%|███████████▏                                                                | 1481/10100 [00:14<01:23, 103.00it/s]





 15%|███████████▏                                                                | 1492/10100 [00:14<01:23, 103.54it/s]





 15%|███████████▎                                                                | 1503/10100 [00:14<01:22, 104.51it/s]





 15%|███████████▍                                                                | 1514/10100 [00:14<01:25, 100.05it/s]







 28%|█████████████████████▌                                                      | 2870/10100 [00:27<01:10, 102.31it/s]





 29%|█████████████████████▉                                                       | 2881/10100 [00:27<01:12, 99.15it/s]





 29%|█████████████████████▊                                                      | 2892/10100 [00:27<01:11, 101.34it/s]





 29%|█████████████████████▊                                                      | 2903/10100 [00:27<01:10, 102.65it/s]





 29%|█████████████████████▉                                                      | 2914/10100 [00:28<01:08, 104.17it/s]





 29%|██████████████████████                                                      | 2925/10100 [00:28<01:08, 105.26it/s]





 29%|██████████████████████                                                      | 2936/10100 [00:28<01:07, 106.04it/s]





 29%|██████████████████████▏                                                     | 2947/10100 [00:28<01:08, 105.07it/s]







 43%|████████████████████████████████▊                                            | 4307/10100 [00:41<00:58, 99.56it/s]





 43%|████████████████████████████████▍                                           | 4318/10100 [00:41<00:56, 101.92it/s]





 43%|████████████████████████████████▌                                           | 4329/10100 [00:41<00:56, 102.78it/s]





 43%|████████████████████████████████▋                                           | 4340/10100 [00:41<00:56, 102.80it/s]





 43%|████████████████████████████████▋                                           | 4351/10100 [00:41<00:55, 104.28it/s]





 43%|████████████████████████████████▊                                           | 4362/10100 [00:41<00:55, 103.85it/s]





 43%|████████████████████████████████▉                                           | 4373/10100 [00:41<00:54, 105.34it/s]





 43%|████████████████████████████████▉                                           | 4384/10100 [00:41<00:53, 106.40it/s]







 57%|███████████████████████████████████████████▊                                 | 5748/10100 [00:54<00:43, 99.98it/s]





 57%|███████████████████████████████████████████▎                                | 5759/10100 [00:55<00:42, 102.23it/s]





 57%|███████████████████████████████████████████▍                                | 5770/10100 [00:55<00:42, 101.85it/s]





 57%|███████████████████████████████████████████▌                                | 5781/10100 [00:55<00:41, 103.89it/s]





 57%|███████████████████████████████████████████▌                                | 5792/10100 [00:55<00:41, 103.87it/s]





 57%|███████████████████████████████████████████▋                                | 5803/10100 [00:55<00:41, 102.41it/s]





 58%|███████████████████████████████████████████▋                                | 5814/10100 [00:55<00:41, 103.12it/s]





 58%|███████████████████████████████████████████▊                                | 5825/10100 [00:55<00:40, 104.51it/s]







 72%|██████████████████████████████████████████████████████▍                     | 7239/10100 [01:08<00:26, 109.28it/s]





 72%|██████████████████████████████████████████████████████▌                     | 7251/10100 [01:08<00:25, 109.84it/s]





 72%|██████████████████████████████████████████████████████▋                     | 7263/10100 [01:09<00:25, 109.93it/s]





 72%|██████████████████████████████████████████████████████▋                     | 7274/10100 [01:09<00:25, 109.64it/s]





 72%|██████████████████████████████████████████████████████▊                     | 7285/10100 [01:09<00:25, 109.44it/s]





 72%|██████████████████████████████████████████████████████▉                     | 7297/10100 [01:09<00:25, 109.96it/s]





 72%|██████████████████████████████████████████████████████▉                     | 7309/10100 [01:09<00:25, 110.93it/s]





 72%|███████████████████████████████████████████████████████                     | 7321/10100 [01:09<00:25, 110.70it/s]







 87%|█████████████████████████████████████████████████████████████████▊          | 8749/10100 [01:22<00:12, 110.90it/s]





 87%|█████████████████████████████████████████████████████████████████▉          | 8761/10100 [01:22<00:12, 110.98it/s]





 87%|██████████████████████████████████████████████████████████████████          | 8773/10100 [01:22<00:11, 111.35it/s]





 87%|██████████████████████████████████████████████████████████████████          | 8785/10100 [01:22<00:11, 109.77it/s]





 87%|██████████████████████████████████████████████████████████████████▏         | 8796/10100 [01:23<00:11, 109.53it/s]





 87%|██████████████████████████████████████████████████████████████████▎         | 8808/10100 [01:23<00:11, 109.71it/s]





 87%|██████████████████████████████████████████████████████████████████▎         | 8820/10100 [01:23<00:11, 110.45it/s]





 87%|██████████████████████████████████████████████████████████████████▍         | 8832/10100 [01:23<00:11, 108.86it/s]







AUC score is :  0.48829900000000004
TP 0
FP 0
TN 100
FN 10000
